## 6.1 Transfer Learning ResNet-34 using Custom Dataset
- Prepare custom image classification dataset in Roboflow 
- Run Transfer Learning ResNet-34
- Evaluate Model performance

⚠️⚠️⚠️ *Please open this notebook in Google Colab* by click below link ⚠️⚠️⚠️<br><br>
<a href="https://colab.research.google.com/github/Muhammad-Yunus/Belajar-Image-Classification/blob/main/Pertemuan%206/6.1%20transfer_learning_resnet_34.ipynb" target="_blank"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a><br><br><br>
- Click `Connect` button in top right Google Colab notebook,<br>
<img src="resource/cl-connect-gpu.png" width="250px">
- If connecting process completed, it will turn to something look like this<br>
<img src="resource/cl-connect-gpu-success.png" width="250px">

- Check GPU connected into Colab environment is active

In [ ]:
!nvidia-smi

#### 6.1.1 Prepare Custom Image Classification Dataset in Roboflow

- Download and extract <font color="orange">Apple2Orange.zip</font> image dataset from https://efrosgans.eecs.berkeley.edu/cyclegan/datasets/apple2orange.zip
- Login to your Roboflow Account https://app.roboflow.com/
- Click <font color="orange">"New Project" button</font>,<br>
<img src="resource/rb0.png" width="300px"><br>
- Input project name <font color="orange">"Fruit Classification"</font> with annotation group <font color="orange">"fruits"</font> and project type <font color="orange">"Classification"</font>, then click <font color="orange">"Create Public Project"</font>, <br>
<img src="resource/rb1.png" width="900px"><br>
- Upload all image on extracted <font color="orange">"apple2orange/"</font> folder to Roboflow by click <font color="orange">"Select Folder"</font> button,<br>
<img src="resource/rb2.png" width="400px"><br>
- Wait untill all images uploaded, <br>
<img src="resource/rb3.png" width="900px"><br>
- Then it should look like below, now click <font color="orange">"Save and Continue"</font> button,<br>
<img src="resource/rb4.png" width="900px"><br>
- Choose <font color="orange">"Split Images Between Train/Valid/Test"</font> with thefolowing portion, then click <font color="orange">"Continue"</font><br>
<img src="resource/rb5.png" width="400px"><br>
- Wait until uploading progress completed, <br>
<img src="resource/rb6.png" width="900px"><br>
- After completed, it will automatically navigating you to the <font color="orange">"Generate"</font> menu on the left side,
- We will choose <font color="orange">Preprocessing - Resize</font> operation to resize image dataset into <font color="orange">224x224</font>,<br>
<img src="resource/rb7.png" width="500px"><br>
- Now it should look like this, then click <font color="orange">"Apply"</font>,<br>
<img src="resource/rb8.png" width="500px"><br>
- You can add augmentation if needed, but for now, we just skip it by click <font color="orange">"Continue"</font>,
- On last section <font color="orange">"Create"</font> just click <font color="orange">"Create"</font>,<br>
<img src="resource/rb9.png" width="500px"><br>
- After that, navigate to menu <font color="orange">"Version"</font> to download dataset script, <br>
<img src="resource/rb10.png" width="900px"><br>
- Click <font color="orange">"Download Dataset"</font>, then choose format <font color="orange">"Folder Structure"</font>, after download script created copy that code, <br>
<img src="resource/rb11.png" width="400px"><br>
- Paste the code bellow to download the dataset,

In [ ]:
# !pip install roboflow

# from roboflow import Roboflow
# rf = Roboflow(api_key="XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX")
# project = rf.workspace("XXXXXXXXXXXXXXX").project("XXXXXXXXXXXXXXXXXXXXXXXX")
# version = project.version(XXXX)
# dataset = version.download("folder")

In [ ]:
dataset.location

<br><br><br><br><br>
#### 6.1.2 Transfer Learning ResNet-34

In [ ]:
import os
import cv2
import torch
from torch.utils.data import Dataset
from torchvision import transforms
import numpy as np


In [ ]:
class CustomDataset(Dataset):
    def __init__(self, images_dir, labels_dir, num_classes):
        self.images_dir = images_dir
        self.labels_dir = labels_dir
        self.num_classes = num_classes
        self.image_files = [f for f in os.listdir(images_dir) if f.endswith('.png')]
        
    def __len__(self):
        # Return the total number of image files
        return len(self.image_files)
    
    def __getitem__(self, idx):
        # Load image using OpenCV
        img_path = os.path.join(self.images_dir, self.image_files[idx])
        image = cv2.imread(img_path)  # OpenCV loads images in BGR format
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)  # Convert to RGB
        
        # Convert to tensor and normalize to 0-1
        image = torch.tensor(image.transpose(2, 0, 1), dtype=torch.float32) / 255.0
        
        # Load label
        label_file = self.image_files[idx].replace('.png', '.txt')
        label_path = os.path.join(self.labels_dir, label_file)
        with open(label_path, 'r') as f:
            label = int(f.read().strip())
        
        # One-hot encode the label
        label_one_hot = np.zeros(self.num_classes, dtype=np.float32)
        label_one_hot[label] = 1.0
        
        # Convert label to PyTorch tensor
        label_tensor = torch.tensor(label_one_hot)
        
        return image, label_tensor

In [ ]:
# Directories where images and labels are stored
num_classes = 2  # Orange and Apple

# Create an instance of CustomDataset
train_set = CustomDataset(images_dir, labels_dir, num_classes, transform=transform)
train_set = CustomDataset(images_dir, labels_dir, num_classes, transform=transform)
train_set = CustomDataset(images_dir, labels_dir, num_classes, transform=transform)

# Create a DataLoader to iterate over the dataset
dataloader = torch.utils.data.DataLoader(dataset, batch_size=32, shuffle=True)

# Iterate over the data
for images, labels in dataloader:
    print(images.shape)  # (batch_size, 3, 224, 224)
    print(labels.shape)  # (batch_size, num_classes)
